In [1]:
import sys
from pathlib import Path

import pandas as pd
import numpy as np
import gtfstk as gt

sys.path.append('../')

import make_gtfs as mg


DATA_DIR = Path('../data')

%load_ext autoreload
%autoreload 2


In [2]:
path = DATA_DIR/'auckland'
pfeed = mg.read_protofeed(path)
feed = mg.build_feed(pfeed)
print(feed)

* agency --------------------
	          agency_name   agency_timezone           agency_url
0  Auckland Transport  Pacific/Auckland  https://at.govt.nz/
* calendar --------------------
	   service_id  monday  tuesday  wednesday  thursday  friday  saturday  sunday  \
0  srv1111100       1        1          1         1       1         0       0   
1  srv0000010       0        0          0         0       0         1       0   

  start_date  end_date  
0   20140101  20150101  
1   20140101  20150101  
* calendar_dates --------------------
	None
* fare_attributes --------------------
	None
* fare_rules --------------------
	None
* feed_info --------------------
	None
* frequencies --------------------
	None
* routes --------------------
	  route_short_name                                    route_long_name  \
0                A  City Link, Wynyard Quarter to Karangahape Rd v...   
4                B  Inner Link. Britomart, Three Lamps, Ponsonby, ...   

   route_type route_id  
0         

In [3]:
f = pd.merge(feed.routes[['route_id', 'route_short_name']], pfeed.frequencies)
f = pd.merge(f, pfeed.service_windows).sort_values('route_id')
f

,route_id,route_short_name,route_long_name,route_type,shape_id,service_window_id,frequency,direction,speed,start_time,end_time,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,rA,A,"City Link, Wynyard Quarter to Karangahape Rd v...",3,shp1,weekday_peak_1,12,1,20.0,07:00:00,09:00:00,1,1,1,1,1,0,0
2,rA,A,"City Link, Wynyard Quarter to Karangahape Rd v...",3,shp1,weekday_peak_2,12,1,20.0,15:00:00,17:00:00,1,1,1,1,1,0,0
4,rA,A,"City Link, Wynyard Quarter to Karangahape Rd v...",3,shp1,weekday_offpeak,4,0,20.0,09:00:00,15:00:00,1,1,1,1,1,0,0
6,rA,A,"City Link, Wynyard Quarter to Karangahape Rd v...",3,shp1,saturday,0,0,20.0,09:00:00,19:00:00,0,0,0,0,0,1,0
1,rB,B,"Inner Link. Britomart, Three Lamps, Ponsonby, ...",2,shp2,weekday_peak_1,6,2,20.0,07:00:00,09:00:00,1,1,1,1,1,0,0
3,rB,B,"Inner Link. Britomart, Three Lamps, Ponsonby, ...",2,shp2,weekday_peak_2,4,2,20.0,15:00:00,17:00:00,1,1,1,1,1,0,0
5,rB,B,"Inner Link. Britomart, Three Lamps, Ponsonby, ...",2,shp2,weekday_offpeak,2,1,20.0,09:00:00,15:00:00,1,1,1,1,1,0,0
7,rB,B,"Inner Link. Britomart, Three Lamps, Ponsonby, ...",2,shp2,saturday,2,0,20.0,09:00:00,19:00:00,0,0,0,0,0,1,0


In [4]:
# Map some trips
prefix = 't-rB-weekday_peak_1-07:00:00'
t = (
    feed.trips
    .loc[lambda x: x.trip_id.str.startswith(prefix)]
)
tids = [t.trip_id.iat[0], t.trip_id.iat[-1]]
feed.map_trips(tids)
    